## Setup

In [1]:
import random
import operator
import pickle
import spacy
import sys
from spacy import displacy
import benepar
import copy
import networkx as nx

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [3]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [4]:
from app.classes.spec.symboleo_spec import Obligation
from app.src.matchers.matchers_builder import MatchersBuilder
from app.src.norm_proposition_updater import NormPropositionUpdater
from app.src.sentence_obligation_converter import SentenceObligationConverter
from app.src.sentence_summarizer import SentenceSummarizer
from app.templates.meat_sale.norms import meat_sale_norms
from app.templates.meat_sale.templates import meat_sale_templates

matchers_builder = MatchersBuilder(nlp)
matchers = matchers_builder.build()
norm_updater = NormPropositionUpdater()

converter = SentenceObligationConverter(nlp, matchers, norm_updater)
summarizer = SentenceSummarizer(nlp)

obligations: list[Obligation] = copy.deepcopy(meat_sale_norms['obligations'])
templates = meat_sale_templates['obligations']

In [43]:
USER_INPUT = True

DEFAULT_OBLIGATION_ID = 'delivery'
DEFAULT_COMPONENT_CHAR = 't'

COMPONENT_DICT = {
    'a': 'antecedent',
    't': 'trigger',
    'c': 'consequent'
}

SAMPLE_SENTENCES = [
    "It is a holiday",
    "The sky is blue",
    "It is not a holiday",
    "It is a full moon",
    "two plus two equals four",
    "the house is on fire",
    "Ted built a house",
    "it is expected to rain",
    "it is raining"
]

DEFAULT_SENTENCE = SAMPLE_SENTENCES[1]


## Gather user input

In [44]:
for ob in obligations:
    print(ob.id)
    print(f'- NL: {templates[ob.id]}')
    print(f'- SYM: {ob.to_sym()}')

delivery
- NL: The Seller shall deliver the Order in one delivery within <delDueDateDays> days to the Buyer at its warehouse
- SYM: delivery: O(seller, buyer, true, WhappensBefore(delivered, delivered.delDueDate))
payment
- NL: The Buyer shall pay <amt> (“amount”) in <curr> (“currency”) to the Seller before <payDueDate>.
- SYM: payment: O(buyer, seller, true, WhappensBefore(paid, paid.payDueDate))
latePayment
- NL: In the event of late payment of the amount owed due, the Buyer shall pay interests equal to <intRate>% of the amount owed
- SYM: latePayment: Happens(Violated(obligations.payment)) -> O(buyer, seller, true, Happens(paidLate))


In [45]:
# Choose the obligation
if USER_INPUT:
    # Choose the obligation
    selected_ob_id = input('Enter obligation id: ')
else:
    selected_ob_id = DEFAULT_OBLIGATION_ID

In [46]:
if USER_INPUT:
    selected_component_char = input('Select the component to update (t/a/c): ')
else:
    selected_component_char = DEFAULT_COMPONENT_CHAR

selected_component = COMPONENT_DICT[selected_component_char]

In [47]:
if USER_INPUT:
    sentence = input('Enter an extension sentence: ')
else:
    sentence = DEFAULT_SENTENCE

## Execution

In [48]:
# summarize the sentence
summarizer.summarize(sentence)


Sentence: buyer has not yet paid

i     TEXT            POS             TAG             DEP             LEMMA           HEAD            ENT            
--------------------------------------------------------------------------------------------------------------
0     buyer           NOUN            NN              nsubj           buyer           paid                           
1     has             AUX             VBZ             aux             have            paid                           
2     not             PART            RB              neg             not             yet                            
3     yet             ADV             RB              advmod          yet             paid                           
4     paid            VERB            VBN             ROOT            pay             paid                           



       S               
   ____|___             
  |        VP          
  |     ___|________    
  NP   |   |  ADVP  VP 
  |    |   |   |    |

In [49]:
obligation = [x for x in obligations if x.id == selected_ob_id][0] 

result = converter.convert(sentence, obligation, selected_component)

print(result.to_sym())

ValueError: 

## Atom Extractor

In [5]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.atom_assembler import AtomAssembler
from app.src.atom_extractor import AtomExtractor
from app.src.dynamic_constructor import DynamicObjectConstructor
from app.src.predicate_builder import PredicateBuilder
from app.src.primitive_checker import PrimitiveChecker
from app.src.primitive_extractor import PrimitiveExtractor
from app.src.graph.graph_builder import GraphBuilder

options = {}
# TODO: Will be adding a lot more args on the primitive extractor
primitive_extractor = PrimitiveExtractor()

graph_builder = GraphBuilder()
digraph = graph_builder.build(PAtom)

primitive_checker = PrimitiveChecker() # leaves should be func parm, not on init

dynamic_constructor = DynamicObjectConstructor(digraph)

predicate_builder = PredicateBuilder(
    digraph,
    primitive_checker,
    dynamic_constructor
)

candidates = [
    'PredicateFunctionHappens',
    'PredicateFunctionHappensWithin',
    'PredicateFunctionOccurs',
    'PredicateFunctionWHappensBefore',
    'PredicateFunctionWHappensBeforeEvent',
    'PredicateFunctionHappensAfter',
]
atom_assembler = AtomAssembler(
    candidates,
    predicate_builder
)

extractor = AtomExtractor(
    options, 
    nlp, 
    primitive_extractor,
    atom_assembler
)

In [6]:
test_sentence = 'the sky is blue'
extractor.extract(test_sentence)

c:\Users\regan\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


[]